In [ ]:
import os
os.chdir(r'C:\Users\Talha Ali\Desktop\machine learning labs')

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
import sys
sys.path.append(r"C:\Users\Talha Ali\Desktop\machine learning labs")

from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input,LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.layers import Input, Reshape, Dense, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [15]:
class PC:
    @staticmethod
    def build(time_steps, num_features, reg=0.0005):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        # define the model input and first CONV module
        inputs = Input(shape=(time_steps, num_features))
        lstm=Bidirectional(LSTM(48,return_sequences=True))(inputs)
        lstm=LSTM(48)(lstm)
        lstm=Activation('tanh')(lstm)
        lstm=Dense(24)(lstm)
        # create the model
        model = Model(inputs, lstm, name="PC")
        
        # return the constructed network architecture
        return model

In [17]:
model1 =  PC.build(time_steps=24, num_features=21, reg=0.005)

In [19]:
model1.summary()

Model: "PC"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 24, 96)              │          26,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 48)                  │          27,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 24)                  │           1,176 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 55,896 (218.34 KB)

 Trainable params: 55,896 (218.34 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import os
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\\E3-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\Talha Ali\Desktop\machine learning labs'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,r"\h2istory.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,r"\h2istory.json"])

In [29]:
model = None
start_epoch = 0
time_steps=24
num_features=21

In [31]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [33]:
# if there is no specific model checkpoint supplied, initialize the network
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

# otherwise, load the checkpoint from disk
elif isinstance(model, str):  # Ensure model is a string path before loading
    if os.path.exists(model):  # Check if the file exists
        print(f"[INFO] loading model from: {model}")
        model = load_model(model)

        # update the learning rate
        print(f"[INFO] old learning rate: {K.get_value(model.optimizer.lr)}")
        K.set_value(model.optimizer.lr, 1e-4)
        print(f"[INFO] new learning rate: {K.get_value(model.optimizer.lr)}")
    else:
        raise FileNotFoundError(f"[ERROR] Model file not found: {model}")

else:
    print("[INFO] Model is already loaded, skipping loading step.")

[INFO] compiling model...


In [ ]:
import os
path_dataset =R'C:\Users\Talha Ali\Desktop\machine learning labs'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\PMLS\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [37]:
time_steps=24
num_features=21

In [39]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=24)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=24)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=24)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.8805935382843018 sec


In [41]:
epochs = 2
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/2
2651/2652 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0627 - mae: 0.0627 - mape: 3112.8538
Epoch 1: val_loss improved from inf to 0.04027, saving model to C:\Users\PMLS\Downloads\ML Lab\checkpoint\ML Lab\lab2\\E3-cp-0001-loss0.04.h5


2652/2652 ━━━━━━━━━━━━━━━━━━━━ 96s 31ms/step - loss: 0.0627 - mae: 0.0627 - mape: 3112.2917 - val_loss: 0.0403 - val_mae: 0.0403 - val_mape: 17.3647
Epoch 2/2
2651/2652 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0370 - mae: 0.0370 - mape: 1896.7234
Epoch 2: val_loss improved from 0.04027 to 0.03646, saving model to C:\Users\PMLS\Downloads\ML Lab\checkpoint\ML Lab\lab2\\E3-cp-0002-loss0.04.h5


2652/2652 ━━━━━━━━━━━━━━━━━━━━ 146s 33ms/step - loss: 0.0370 - mae: 0.0370 - mape: 1896.9467 - val_loss: 0.0365 - val_mae: 0.0365 - val_mape: 14.8270


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError

# Corrected file path
model_path = r"C:C:\Users\Talha Ali\Desktop\machine learning labs\E3-cp-0002-loss0.04.h5"

# Verify if the file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Error: File not found at {model_path}")

# Load model with custom metric handling
try:
    model = load_model(model_path, custom_objects={'mae': MeanAbsoluteError()})
    print("Model loaded successfully with custom metric!")
except TypeError:
    print("Warning: Could not locate custom function. Loading without compilation.")
    model = load_model(model_path, compile=False)

# Predictions
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Error metrics
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE):', round(MAE, 2))

MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE):', round(MEDAE, 2))

MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE):', round(MSE, 2))

RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE):', round(RMSE, 2))

MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred) / y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE):', round(MAPE, 2), '%')

MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred) / y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE):', round(MDAPE, 2), '%')

print('\n\ny_test_unscaled.shape =', y_test_unscaled.shape)
print('y_pred.shape =', y_pred.shape)


Model loaded successfully with custom metric!
378/378 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step
Mean Absolute Error (MAE): 583.74
Median Absolute Error (MedAE): 434.46
Mean Squared Error (MSE): 614651.04
Root Mean Squared Error (RMSE): 784.0
Mean Absolute Percentage Error (MAPE): 3.9 %
Median Absolute Percentage Error (MDAPE): 3.02 %


y_test_unscaled.shape = (12082, 24)
y_pred.shape = (12082, 24)


In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\Talha Ali\Desktop\machine learning labs\E3-cp-0001-loss0.04.h5'
start_epoch= 17

In [ ]:
import os
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.saving import register_keras_serializable

# 🔹 Register custom metrics
@register_keras_serializable(package="Custom", name="mae")
def mae(y_true, y_pred):
    return MeanAbsoluteError()(y_true, y_pred)

@register_keras_serializable(package="Custom", name="mape")
def mape(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / K.clip(y_true, K.epsilon(), None))) * 100

# Correct the checkpoint file path
checkpoint_path = r"C:\Users\Talha Ali\Desktop\machine learning labs checkpoint\ML Lab\lab2\E3-cp-0002-loss0.04.h5"

# Ensure the file exists before loading
if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"❌ Error: Model checkpoint not found at {checkpoint_path}")

print(f"[INFO] Loading model from {checkpoint_path}...")


try:
    model = load_model(checkpoint_path, custom_objects={"mae": mae, "mape": mape}, compile=False)
    print("Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

#  Recompile the model (since `compile=False` was used)
print("[INFO] Recompiling the model with Adam optimizer...")
optimizer = Adam(learning_rate=1e-4)
model.compile(loss="mae", optimizer=optimizer, metrics=["mae", "mape"])

# Verify the learning rate
print("[INFO] New learning rate:", K.get_value(model.optimizer.learning_rate))

#  Verify model structure
print(model.summary())


[INFO] Loading model from C:\Users\PMLS\Downloads\ML Lab\checkpoint\ML Lab\lab2\E3-cp-0002-loss0.04.h5...
✅ Model loaded successfully!
[INFO] Recompiling the model with Adam optimizer...
[INFO] New learning rate: 1e-04


Model: "PC"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 24, 96)              │          26,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 48)                  │          27,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 24)                  │           1,176 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 55,896 (218.34 KB)

 Trainable params: 55,896 (218.34 KB)

 Non-trainable params: 0 (0.00 B)

None


In [66]:
epochs = 2
verbose = 1 #0
batch_size = 128
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/2
663/663 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0319 - mae: 0.0319 - mape: 2123.0183
Epoch 1: val_loss improved from inf to 0.03384, saving model to C:\Users\PMLS\Downloads\ML Lab\checkpoint\ML Lab\lab2\E3-cp-0002-loss0.04.h5


663/663 ━━━━━━━━━━━━━━━━━━━━ 55s 65ms/step - loss: 0.0319 - mae: 0.0319 - mape: 2123.1499 - val_loss: 0.0338 - val_mae: 0.0338 - val_mape: 13.8051
Epoch 2/2
663/663 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0313 - mae: 0.0313 - mape: 2023.0042
Epoch 2: val_loss improved from 0.03384 to 0.03374, saving model to C:\Users\PMLS\Downloads\ML Lab\checkpoint\ML Lab\lab2\E3-cp-0002-loss0.04.h5


663/663 ━━━━━━━━━━━━━━━━━━━━ 44s 67ms/step - loss: 0.0313 - mae: 0.0313 - mape: 2023.2651 - val_loss: 0.0337 - val_mae: 0.0337 - val_mape: 13.7826


In [ ]:
import os
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.saving import register_keras_serializable

# 🔹 Register custom metrics
@register_keras_serializable(package="Custom", name="mae")
def mae(y_true, y_pred):
    return MeanAbsoluteError()(y_true, y_pred)

@register_keras_serializable(package="Custom", name="mape")
def mape(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / K.clip(y_true, K.epsilon(), None))) * 100

#  Correct checkpoint file path
checkpoint_path = r"C:\Users\Talha Ali\Desktop\machine learning labs\checkpoint\\E3-cp-0002-loss0.04.h5"

# Ensure the file exists before loading
if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f" Error: Model checkpoint not found at {checkpoint_path}")

print(f"[INFO] Loading model from {checkpoint_path}...")

#  Load the model with custom objects
try:
    model = load_model(checkpoint_path, custom_objects={"mae": mae, "mape": mape}, compile=False)
    print(" Model loaded successfully!")
except Exception as e:
    print(f" Error loading model: {e}")
    exit()

# Recompile the model (since `compile=False` was used)
print("[INFO] Recompiling the model with Adam optimizer...")
optimizer = Adam(learning_rate=1e-4)
model.compile(loss="mae", optimizer=optimizer, metrics=["mae", "mape"])

#  Verify model structure
print(model.summary())

# ========================= PREDICTIONS & METRICS =========================

# Ensure `test_X` and `test_y` are defined before using them
if "test_X" not in locals() or "test_y" not in locals():
    raise ValueError(" Error: `test_X` or `test_y` is not defined. Ensure they are initialized.")

# Make predictions
y_pred_scaled = model.predict(test_X)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Compute evaluation metrics
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
MEDAE = np.median(abs(y_pred - y_test_unscaled))
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred) / y_test_unscaled))) * 100
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred) / y_test_unscaled))) * 100

# Print results
print('Mean Absolute Error (MAE):', np.round(MAE, 2))
print('Median Absolute Error (MedAE):', np.round(MEDAE, 2))
print('Mean Squared Error (MSE):', np.round(MSE, 2))
print('Root Mean Squared Error (RMSE):', np.round(RMSE, 2))
print('Mean Absolute Percentage Error (MAPE):', np.round(MAPE, 2), '%')
print('Median Absolute Percentage Error (MDAPE):', np.round(MDAPE, 2), '%')

print('\n\ny_test_unscaled.shape:', y_test_unscaled.shape)
print('y_pred.shape:', y_pred.shape)


[INFO] Loading model from C:\Users\PMLS\Downloads\ML Lab\checkpoint\ML Lab\lab2\E3-cp-0002-loss0.04.h5...
 Model loaded successfully!
[INFO] Recompiling the model with Adam optimizer...


Model: "PC"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 24, 96)              │          26,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 48)                  │          27,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 24)                  │           1,176 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 55,896 (218.34 KB)

 Trainable params: 55,896 (218.34 KB)

 Non-trainable params: 0 (0.00 B)

None
378/378 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step
Mean Absolute Error (MAE): 549.04
Median Absolute Error (MedAE): 404.43
Mean Squared Error (MSE): 551068.79
Root Mean Squared Error (RMSE): 742.34
Mean Absolute Percentage Error (MAPE): 3.68 %
Median Absolute Percentage Error (MDAPE): 2.81 %


y_test_unscaled.shape: (12082, 24)
y_pred.shape: (12082, 24)
